![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

# Importaciones

In [19]:

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")


# Carga de datos

In [20]:

df_rental = pd.read_csv("rental_info.csv")
df_rental.head()


,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401


# Ingeniería de características

In [21]:

# 1) Calcular duración en días
df_rental["rental_length"] = (
    pd.to_datetime(df_rental["return_date"])
    - pd.to_datetime(df_rental["rental_date"])
)
df_rental["rental_length_days"] = df_rental["rental_length"].dt.days

# 2) Dummies de features especiales
df_rental["deleted_scenes"] = np.where(
    df_rental["special_features"].str.contains("Deleted Scenes"), 1, 0
)
df_rental["behind_the_scenes"] = np.where(
    df_rental["special_features"].str.contains("Behind the Scenes"), 1, 0
)

df_rental.head()


,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2,rental_length,rental_length_days,deleted_scenes,behind_the_scenes
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,3 days 20:46:00,3,0,1
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,2 days 20:05:00,2,0,1
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,7 days 05:44:00,7,0,1
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,2 days 02:24:00,2,0,1
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,4 days 01:05:00,4,0,1


# Preparar X e y

In [22]:

cols_to_drop = [
    "special_features", "rental_length", 
    "rental_length_days", "rental_date", "return_date"
]
X = df_rental.drop(cols_to_drop, axis=1)
y = df_rental["rental_length_days"]


# Split Train/Test

In [23]:

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=9
)


# Selección de características con Lasso

In [24]:

lasso = Lasso(alpha=0.3, random_state=9)
lasso.fit(X_train, y_train)

# Filtrar columnas con coeficiente > 0
mask = lasso.coef_ > 0
X_lasso_train = X_train.loc[:, mask]
X_lasso_test  = X_test.loc[:,  mask]


# Regresión Lineal sobre las características seleccionadas

In [25]:

ols = LinearRegression().fit(X_lasso_train, y_train)
y_pred_ols = ols.predict(X_lasso_test)
mse_ols = mean_squared_error(y_test, y_pred_ols)
print(f"MSE Regresión Lineal (features Lasso): {mse_ols:.3f}")


MSE Regresión Lineal (features Lasso): 4.812


# Búsqueda de hiperparámetros para Random Forest

In [26]:

param_dist = {
    "n_estimators": np.arange(10, 201, 10),
    "max_depth": np.arange(2, 21, 2)
}
rf = RandomForestRegressor(random_state=9)
rand_search = RandomizedSearchCV(
    rf, param_dist, cv=5, n_iter=30, random_state=9
)
rand_search.fit(X_train, y_train)

best_params = rand_search.best_params_
print("Mejores hiperparámetros RF:", best_params)


Mejores hiperparámetros RF: {'n_estimators': 120, 'max_depth': 14}


# Entrenar Random Forest con los mejores hiperparámetros

In [27]:

rf_best = RandomForestRegressor(
    n_estimators=best_params["n_estimators"],
    max_depth=best_params["max_depth"],
    random_state=9
).fit(X_train, y_train)

y_pred_rf = rf_best.predict(X_test)
mse_rf = mean_squared_error(y_test, y_pred_rf)
print(f"MSE Random Forest: {mse_rf:.3f}")


MSE Random Forest: 2.034


# Selección final de modelo

In [28]:

best_model = rf_best
best_mse   = mse_rf
